# 09. Computing coarse-grained molecular features

This notebook shows how to compute pairwise ditances, angles and dihedrals between CG beads given a CG mapping. The CG mapping used in this example is generated from [DSGPM](https://github.com/rochesterxugroup/DSGPM) model. 
You must need MDAnalysis and NetworkX in your working environment to run this example. 

In [1]:
import hoomd
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda
import numpy as np
# disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import warnings
warnings.filterwarnings('ignore')

2 Physical GPUs, 2 Logical GPUs


## Building the model

This model uses two MDAnalysis universes with and without hydrogens as the DSGPM model only maps heavy atoms of a given molecule. Hence, we have to add the missing hydrogen atoms to the corresponding CG beads. This model only reads from the trajectory and does not output any forces.    

In [9]:
class CGModel(htf.SimModel):
    
    def setup(self):
        self.avg_cgr = tf.keras.metrics.MeanTensor()
        self.avg_cga = tf.keras.metrics.MeanTensor()
        self.avg_cgd = tf.keras.metrics.MeanTensor()
    
    def compute(self):
        cg_fts = []
        r_tensor = []
        a_tensor = []
        d_tensor = []
        
        directory = os.getcwd()
        jfile = os.path.join(directory,'ex9_cgmap_segA.json')
        
        #mda universe without H's
        u2 = mda.Universe(os.path.join(directory,'ex9_segA_xH.pdb'))
        #mda universe with H's
        u1 = mda.Universe(os.path.join(directory,'ex9_segA.pdb'))
        
        #get CG bead indices that make bonds, angles, dihedrals and
        #CG coordinates
        cg_fts = htf.compute_cg_graph(DSGPM=True,infile=jfile,group_atoms=True,
                                      u_no_H=u2, u_H=u1)
        
        for i in range(len(cg_fts[0])):
            cg_r = htf.mol_bond_distance(CG = True, cg_positions = cg_fts[-1],
                                         b1=cg_fts[0][i][0],b2=cg_fts[0][i][1])
            r_tensor.append(cg_r)
        
        for j in range(len(cg_fts[1])):       
            cg_a = htf.mol_angle(CG= True, cg_positions=cg_fts[-1],
                                 b1=cg_fts[1][j][0],b2=cg_fts[1][j][1],b3=cg_fts[1][j][2])
            a_tensor.append(cg_a)
        
        
        for k in range(len(cg_fts[2])):
            cg_d = htf.mol_dihedral(CG=True,cg_positions=cg_fts[-1], 
                                    b1=cg_fts[2][k][0],b2=cg_fts[2][k][1],b3=cg_fts[2][k][2],b4=cg_fts[2][k][3])
            d_tensor.append(cg_d)
        
        self.avg_cgr.update_state(r_tensor)
        self.avg_cga.update_state(a_tensor)
        self.avg_cgd.update_state(d_tensor)
        
        return cg_fts
    
cgmodel = CGModel(16,output_forces=False)

## Read frames from the trajectory

In [10]:
universe = mda.Universe('ex9_segA.pdb','ex9_segA.trr')

In [11]:
for inputs, ts in htf.iter_from_trajectory(16, universe, r_cut=10):
    result = cgmodel(inputs)

In [12]:
cgR = cgmodel.avg_cgr.result().numpy()
cgD = cgmodel.avg_cgd.result().numpy()*180./np.pi
cgA = cgmodel.avg_cga.result().numpy()*180./np.pi
print('CG pairwise distances:',cgR,'\n')
print('CG angles:',cgA,'\n')
print('CG dihedral angles:',cgD)

CG pairwise distances: [ 5.444767   1.1312735  6.8374457  2.9382575  2.4656727  4.441667
  3.1997018  4.215074   3.5845394  2.1536508  7.9029527  3.8829803
  6.7588325  6.4775     2.2552814  4.9249864 15.143307 ] 

CG angles: [ 57.06854   75.2243    83.65678  113.90824   30.891647  61.17423
  40.556572  27.59399   50.535973 149.74559   46.74406   91.21443
  44.42902  157.15245   45.615276 121.53178  140.93127   90.67872
  51.73296  156.72931 ] 

CG dihedral angles: [ 61.195976 177.25414    4.78611  111.418495 176.0712   133.15433
  84.99487  135.76794  147.14043    4.834293 168.74007  124.28044
 175.61665   21.14613  163.78847   32.634525   9.021179 175.17896
  10.56541    7.195493]
